In [32]:
## imports 
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
import pandas as pd
import csv

In [33]:
from bs4 import BeautifulSoup
import requests
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
# all_sentences = []
# with open('Data/personification.txt','r') as f:
#     all_sentences = f.readlines()
# all_sentences += df['s'].tolist()

In [4]:
# len(all_sentences)

In [5]:
# reverse_lines = []
# for string in all_sentences:
#     s = string.split()[::-1]
#     l = []
#     for i in s:
#         l.append(i)
#     if(len(l) > 4):
#         reverse_lines.append(" ".join(l))
# with open("Data/person_rev.txt",'w', encoding='utf-8') as f:
#     for line in reverse_lines:
#             f.write(line + '\n')

In [22]:
with open('norm_rev.txt','r',encoding="utf8") as story:
  story_data = story.readlines()
len(story_data)

2813551

In [23]:
story_data[:-100]

['salts. and wines are tears Barrels\n',
 'tails! goody on whisk a With\n',
 'heads. the fix to maces Wiggle\n',
 'ceased. are boxes on jack in Heads\n',
 'bosses. truly paranoid to Cry\n',
 'boys. your take jokers are Bosses\n',
 'apples. fire with bogs are Studs\n',
 'bands. badly in wash Descents\n',
 'cats. with smart are sales Wholly\n',
 'China? in honors tenth got Who\n',
 'plays! and play to grand Homage\n',
 'cry. then hearts of times the Trim\n',
 'wail. voice but steels in Tanks\n',
 'whisked. that tails by dragged Bossy\n',
 'wise. the love and timid very Go\n',
 'ends. are laws but lent are Hands\n',
 'lands. borrowed are courts on Cases\n',
 'retch! to treads with long Length\n',
 'here. watch and times on Straps\n',
 'men. are all but tanks Arrays\n',
 'ends. the steal suctions all Cross\n',
 'objects. on cages are minds on Cave\n',
 'wounds. our stop to curses Confine\n',
 'grounds. on jump and bodies your Whirl\n',
 'flings. with kicks after soldiers of Crouch\n',
 'mi

Cleaning Process 

In [24]:
import re                                

# Regular expressions to use sub function for replacing the useless text from the data

def clean_text(text):
  text = re.sub(r',', '', text)
  text = re.sub(r'\'', '',  text)
  text = re.sub(r'\"', '', text)
  text = re.sub(r'\(', '', text)
  text = re.sub(r'\)', '', text)
  text = re.sub(r'\n', '', text)
  text = re.sub(r'“', '', text)
  text = re.sub(r'”', '', text)
  text = re.sub(r'’', '', text)
  text = re.sub(r'\.', '', text)
  text = re.sub(r';', '', text)
  text = re.sub(r':', '', text)
  text = re.sub(r'\-', '', text)

  return text

In [25]:
# lower_data = story_data.lower()
# split_data = lower_data.splitlines()     
# Splitting the data to get every line seperately but this will give the list of uncleaned data

# print(split_data[0])                         

final_data = []                               
# initiating a argument with blank string to hold the values of final cleaned data

for line in story_data[:10000]:
    line = line.lower()
    line = clean_text(line)
    final_data.append(line)

# print(final)
# final_data = final.split('\n')       # splitting again to get list of cleaned and splitted data ready to be processed
print(len(final_data))

10000


In [26]:
max_vocab = 1000000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(final_data)

# Getting the total number of words of the data.
word2idx = tokenizer.word_index
print(len(word2idx))
print(word2idx)
vocab_size = len(word2idx) + 1        # Adding 1 to the vocab_size because the index starts from 1 not 0. This will make it uniform when using it further
print(vocab_size)

9428
{'the': 1, 'and': 2, 'of': 3, 'to': 4, 'in': 5, 'with': 6, 'his': 7, 'a': 8, 'he': 9, 'all': 10, 'from': 11, 'that': 12, 'on': 13, 'as': 14, 'i': 15, 'him': 16, 'for': 17, 'or': 18, 'but': 19, 'not': 20, 'they': 21, 'by': 22, 'was': 23, 'is': 24, 'it': 25, 'this': 26, 'you': 27, 'my': 28, 'at': 29, 'her': 30, 'so': 31, 'then': 32, 'be': 33, 'them': 34, 'me': 35, 'when': 36, 'their': 37, 'what': 38, 'our': 39, 'are': 40, 'thir': 41, 'like': 42, 'through': 43, 'who': 44, 'no': 45, 'more': 46, 'now': 47, 'we': 48, 'thy': 49, 'heavn': 50, 'thou': 51, 'your': 52, 'thus': 53, 'will': 54, 'shall': 55, 'had': 56, 'us': 57, 'th': 58, 'have': 59, 'where': 60, 'into': 61, 'great': 62, 'upon': 63, 'were': 64, 'which': 65, 'water': 66, 'till': 67, 'there': 68, 'if': 69, 'one': 70, 'o': 71, 'these': 72, 'old': 73, 'thee': 74, 'whom': 75, 'hiawatha': 76, 'nor': 77, 'can': 78, 'yet': 79, 'though': 80, 'down': 81, 'said': 82, 'earth': 83, 'how': 84, 'man': 85, 'made': 86, 'among': 87, 'its': 88, '

In [27]:
input_seq = []

for line in final_data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_seq = token_list[:i+1]
    input_seq.append(n_gram_seq)

print(input_seq)

[[4506, 2], [4506, 2, 3005], [4506, 2, 3005, 40], [4506, 2, 3005, 40, 802], [4506, 2, 3005, 40, 802, 4507], [1817, 4508], [1817, 4508, 13], [1817, 4508, 13, 4509], [1817, 4508, 13, 4509, 8], [1817, 4508, 13, 4509, 8, 6], [803, 1], [803, 1, 3006], [803, 1, 3006, 4], [803, 1, 3006, 4, 4510], [803, 1, 3006, 4, 4510, 4511], [1130, 40], [1130, 40, 4512], [1130, 40, 4512, 13], [1130, 40, 4512, 13, 1818], [1130, 40, 4512, 13, 1818, 5], [1130, 40, 4512, 13, 1818, 5, 803], [3007, 3008], [3007, 3008, 4513], [3007, 3008, 4513, 4], [3007, 3008, 4513, 4, 328], [1131, 52], [1131, 52, 262], [1131, 52, 262, 4514], [1131, 52, 262, 4514, 40], [1131, 52, 262, 4514, 40, 3007], [2259, 137], [2259, 137, 6], [2259, 137, 6, 3009], [2259, 137, 6, 3009, 40], [2259, 137, 6, 3009, 40, 4515], [804, 4516], [804, 4516, 5], [804, 4516, 5, 1132], [804, 4516, 5, 1132, 4517], [4518, 6], [4518, 6, 2260], [4518, 6, 2260, 40], [4518, 6, 2260, 40, 4519], [4518, 6, 2260, 40, 4519, 1524], [4520, 5], [4520, 5, 3010], [4520, 5,

In [30]:
# Getting the maximum length of sequence for padding purpose
max_seq_length = max(len(x) for x in input_seq)
print(max_seq_length)
# Padding the sequences and converting them to array
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_length, padding='pre'))
print(input_seq)

27
[[   0    0    0 ...    0 4506    2]
 [   0    0    0 ... 4506    2 3005]
 [   0    0    0 ...    2 3005   40]
 ...
 [   0    0    0 ...   32  522    5]
 [   0    0    0 ...  522    5   78]
 [   0    0    0 ...    5   78   44]]


In [13]:
xs = input_seq[:, :-1]        # xs contains every word in sentence except the last one because we are using this value to predict the y value
labels = input_seq[:, -1]     # labels contains only the last word of the sentence which will help in hot encoding the y value in next step
print("xs: ",xs)
print("labels:",labels)

xs:  [[   0    0    0 ...    0    0 4506]
 [   0    0    0 ...    0 4506    2]
 [   0    0    0 ... 4506    2 3005]
 ...
 [   0    0    0 ...   18   32  522]
 [   0    0    0 ...   32  522    5]
 [   0    0    0 ...  522    5   78]]
labels: [   2 3005   40 ...    5   78   44]


In [14]:
from tensorflow.keras.utils import to_categorical

# one-hot encoding the labels according to the vocab size

# The matrix is square matrix of the size of vocab_size. Each row will denote a label and it will have 
# a single +ve value(i.e 1) for that label and other values will be zero. 

ys = to_categorical(labels, num_classes=vocab_size)
print(ys)

[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


In [16]:
# # using the functional APIs of keras to define the model
# i = Input(shape=(max_seq_length - 1, ))                           
# # using 1 less value becasuse we are preserving the last value for predicted word 
# x = Embedding(vocab_size, 124)(i)
# x = Dropout(0.2)(x)
# x = LSTM(520, return_sequences=True)(x)
# x = Bidirectional(layer=LSTM(340, return_sequences=True))(x)
# x = GlobalMaxPooling1D()(x)
# x = Dense(1024, activation='relu')(x)
# x = Dense(vocab_size, activation='softmax')(x)

# model = Model(i,x)

In [17]:
# using the functional APIs of keras to define the model
i = Input(shape=(max_seq_length - 1, ))                           
# using 1 less value becasuse we are preserving the last value for predicted word 
x = Embedding(vocab_size, 124)(i)
x = Dropout(0.2)(x)
x = LSTM(1024, return_sequences=True)(x)
x = Bidirectional(layer=LSTM(340, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1280, activation='relu')(x)
x = Dense(vocab_size, activation='softmax')(x)

model = Model(i,x)

In [18]:
model.compile(optimizer=Adam(lr=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])
model.summary()    

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26)]              0         
                                                                 
 embedding (Embedding)       (None, 26, 124)           1169196   
                                                                 
 dropout (Dropout)           (None, 26, 124)           0         
                                                                 
 lstm (LSTM)                 (None, 26, 1024)          4706304   
                                                                 
 bidirectional (Bidirectiona  (None, 26, 680)          3712800   
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 680)              0         
 lMaxPooling1D)                                              

C:\Users\rudra\anaconda3\envs\run-in-gpu\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [19]:
r = model.fit(xs,ys,epochs=50)

Epoch 1/50
1839/1839 [==============================] - 82s 40ms/step - loss: 6.7385 - accuracy: 0.0749
Epoch 2/50
1839/1839 [==============================] - 104s 57ms/step - loss: 6.3846 - accuracy: 0.0955
Epoch 3/50
1839/1839 [==============================] - 116s 63ms/step - loss: 6.1027 - accuracy: 0.1108
Epoch 4/50
1359/1839 [=====================>........] - ETA: 35s - loss: 5.8899 - accuracy: 0.1203

In [ ]:
import matplotlib.pyplot as plt
plt.plot(r.history['accuracy'])

In [47]:
def predict_words(seed, no_words):
  for i in range(no_words):
    token_list = tokenizer.texts_to_sequences([seed])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=1)
    print(predicted)

#     new_word = ''
#     new_word = {i for i in tokenizer.word_index if tokenizer.word_index[i]==predicted}[0]
    for word, index in tokenizer.word_index.items():
      if predicted == index:
        new_word = word
        break
    seed += " " + new_word
#     print(seed)
    if(i==no_words-1):
        return seed
    



In [48]:
seed_text = 'boy'
next_words = 10
sentence = predict_words(seed_text, next_words)
words = sentence.split()
reversed_sentence = " ".join(words[::-1])
print(reversed_sentence)


1/1 [==============================] - 0s 28ms/step
[428]
1/1 [==============================] - 0s 24ms/step
[1]
1/1 [==============================] - 0s 23ms/step
[3]
1/1 [==============================] - 0s 25ms/step
[1623]
1/1 [==============================] - 0s 24ms/step
[1]
1/1 [==============================] - 0s 23ms/step
[4]
1/1 [==============================] - 0s 27ms/step
[2]
1/1 [==============================] - 0s 24ms/step
[215]
1/1 [==============================] - 0s 24ms/step
[626]
1/1 [==============================] - 0s 24ms/step
[146]
both put off and to the warcry of the human boy


In [ ]:
model.save('new_sen_generator.h5') # Will create a HDF5 file of the model

In [29]:
import pickle
with open('norm_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [44]:
model = keras.models.load_model('models/norm_generator.h5')
with open('tokenizers/norm_tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

In [35]:
from tensorflow import keras

In [46]:
max_seq_length

27